# COMMUNICATION Summary

集群场景通信算子数据分析

主要包含以下3个统计内容：
1. 按算子类型分组的，整个集群通信算子耗时的统计情况
2. 按算子类型分组的，每个Rank上通信算子的耗时情况
3. 整个集群平均耗时最久的TOP通信算子

### 数据准备

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

import plotly.offline as pyo

def is_lab_notebook():
    import re
    import psutil
    return any(re.search('jupyter--lab-script', x) for x in psutil.Process().parent().cmdline())

if is_lab_notebook():
    pyo.init_notebook_mode()

import pandas as pd
pd.options.plotting.backend = "plotly"
pd.set_option("display.max_rows", 100)
pd.set_option("display.width", 1000)

import cluster_display

all_stats_df = pd.read_csv("all_stats.csv", index_col="OpType")
rank_stats_df = pd.read_csv("rank_stats.csv", index_col="OpType")
top_op_stats_df = pd.read_csv("top_op_stats.csv", index_col="OpName")

### 集群通信算子耗时分析

将整个集群所有Rank的通信算子进行汇总，按算子类型分类，统计分析耗时情况，时间单位为微秒(us)

包含以下统计项：
- Count：算子数量
- Mean：平均耗时
- Std：标准差
- Min：最小值
- Q1：四分之一分位数
- Median：中位数
- Q3：四分之三分位数
- Max：最大值
- Sum：总耗时

In [ ]:
display(all_stats_df)
fig_all_rank = cluster_display.display_duration_boxplots(None, all_stats_df, x_title="Hccl OpType")
fig_per_rank = cluster_display.display_graph(None, all_stats_df.index, all_stats_df[["Q1(Us)", "Median(Us)", "Q3(Us)"]], title="50% of Distribution", x_title="Hccl OpType")

### 集群Rank通信算子耗时分析

将集群内每个Rank的通信算子进行汇总，按算子类型分类，统计分析耗时情况，时间单位为微秒(us)

包含以下统计项：
- Count：算子数量
- Mean：平均耗时
- Std：标准差
- Min：最小值
- Q1：四分之一分位数
- Median：中位数
- Q3：四分之三分位数
- Max：最大值
- Sum：总耗时

In [ ]:
rank_stats_gdf = rank_stats_df.groupby(rank_stats_df.index)
cluster_display.display_stats_per_rank_groups_combobox(rank_stats_gdf)

### 集群TOP-N通信算子耗时分析

统计集群内耗时最多的TOP-N通信算子，时间单位为微秒(us)

包含以下统计项：
- Count：算子数量
- Mean：平均耗时
- Std：标准差
- Min：最小值
- Q1：四分之一分位数
- Median：中位数
- Q3：四分之三分位数
- Max：最大值
- Sum：总耗时
- MinRank：耗时最少算子所在的Rank
- MaxRank：耗时最长算子所在的Rank

In [ ]:
display(top_op_stats_df)
fig_top_op = cluster_display.display_duration_boxplots(None, top_op_stats_df, x_title="Hccl OpName")